# Управление

In [ ]:
import numpy as np
import scipy.linalg as spl
import scipy.optimize as sopt
import scipy.stats as stats

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
#from torch.autograd.functional import jacobian

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

font = {'family' : 'Liberation Sans',
        'weight' : 'normal',
        'size'   : 30}

matplotlib.rc('font', **font)

### Модуль для управления

In [ ]:
class TestControl(nn.Module):
    time = 10.0
    N = 100
    
    def __init__(self):
        super(TestControl, self).__init__()
        self.dt = self.time / self.N
        self.control = nn.Linear(2, 4)
        torch.nn.init.uniform_(self.control.weight, 0.0, 0.01)
        
    def forward(self, x_0, params):
        X = [x_0]
        C = []
        for step in range(self.N):
            C.append(self.control(X[-1]))
            X.append(X[-1] + self.dt *
                     (torch.bmm(torch.reshape(C[-1], (x_0.shape[0],2,2)),
                                X[-1].unsqueeze(dim=-1)).squeeze(dim=-1) + params))
            
        return torch.stack(X, dim=1), torch.stack(C, dim=1)

In [ ]:
cont = TestControl()

In [ ]:
x_0 = np.array([[0.0, 0.0], [1.0, 1.0]])
p_0 = np.array([[1.0, 1.0], [0.0, 0.0]])

In [ ]:
cont.forward(torch.tensor(x_0).float(), torch.tensor(p_0).float())

In [ ]:
class IVPDataset(Dataset):
    def __init__(self, x_shape, p_shape):
        self.x_shape = x_shape
        self.p_shape = p_shape
        
        self.x_0_dist = stats.norm()
        self.p_0_dist = stats.norm()
    
    def __len__(self):
        return 10000
    
    def __getitem__(self, idx):
        return self.x_0_dist.rvs(self.x_shape).astype(np.float32), self.p_0_dist.rvs(self.p_shape).astype(np.float32)

In [ ]:
dataset = IVPDataset((2), (2))
dataset[0]

In [ ]:
dataloader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
 )

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(cont.parameters(), lr=0.0001, momentum=0.9)

In [ ]:
def my_loss(X, C):
    loss = torch.mean((X)**2) + torch.mean((C)**2)
    return loss

In [ ]:
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        X_0, P_0 = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        X, C = cont(X_0, P_0)
        loss = my_loss(X, C)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
cont.forward(torch.tensor(x_0).float(), torch.tensor(p_0).float())